In [1]:
!pip uninstall --yes nvidia-ml-py3
!pip uninstall --yes pandas-profiling
!pip uninstall --yes scipy
!pip install scipy==1.7.2
!pip install pandas-profiling
!pip install auto-sklearn==0.14.0

Found existing installation: pandas-profiling 3.1.0
Uninstalling pandas-profiling-3.1.0:
  Successfully uninstalled pandas-profiling-3.1.0
Found existing installation: scipy 1.7.2
Uninstalling scipy-1.7.2:
  Successfully uninstalled scipy-1.7.2
  Using cached scipy-1.7.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (38.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl (261 kB)


In [34]:
import pandas as pd
import sklearn.model_selection
from sklearn.preprocessing import LabelEncoder

# read train set
data_train = pd.read_csv('data_train.csv')

# preprocess
# replace nan values with zeros
data_train.fillna(0)

# convert object dtypes to categorial
obj_columns = data_train.select_dtypes(['object']).columns
for col in obj_columns:
    data_train[col] = data_train[col].astype('category')

X = data_train.drop("class", 1) # Feature Matrix
y = data_train["class"]         # Labels

# split data into train and test sets
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size=0.33, random_state=3
)

            V1           V2           V3  ...         V98  V99  V100
0  1819.870220 -1049.683290 -1097.551530  ...  863.381036    D     D
1  5338.339092  -896.336093  -169.203551  ...  449.478649    B     D
2   376.999942  -457.473122   -12.817029  ...  417.676998    D     D
3  6633.434350 -1139.578274   562.928815  ...  502.876430    D     D
4 -3147.858218 -1315.076756 -1640.884995  ...  377.266730    D     D

[5 rows x 100 columns]
0    DO
1    AT
2    DZ
3    EF
4    DW
Name: class, dtype: category
Categories (100, object): ['AB', 'AC', 'AD', 'AE', ..., 'EH', 'EI', 'EJ', 'EK']
V1       float64
V2       float64
V3       float64
V4       float64
V5       float64
          ...   
V96      float64
V97      float64
V98      float64
V99     category
V100    category
Length: 100, dtype: object


In [35]:
import autosklearn.classification
import autosklearn.metrics

# Create and train an ensemble with AutoML 
estimator_askl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=15*60, # in seconds
    resampling_strategy='holdout-iterative-fit',
    memory_limit=4096,
    metric=autosklearn.metrics.accuracy,
    n_jobs=-1,
    include={
        'classifier': ['random_forest', 'adaboost', 'gaussian_nb', 'sgd', 'gradient_boosting']
    },
)

estimator_askl.fit(X=X_train, y=y_train, dataset_name='unknown', X_test=X_test, y_test=y_test)

/usr/local/lib/python3.7/dist-packages/distributed/node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45617 instead
  expected, actual


AutoSklearnClassifier(include={'classifier': ['random_forest', 'adaboost',
                                              'gaussian_nb', 'sgd',
                                              'gradient_boosting']},
                      memory_limit=4096, metric=accuracy, n_jobs=-1,
                      per_run_time_limit=180,
                      resampling_strategy='holdout-iterative-fit',
                      time_left_for_this_task=900)

In [36]:
from sklearn.metrics import accuracy_score

# predict on dataset 
predictions = estimator_askl.predict(X)
print('Predictions',predictions)
print('Expected', y)
performance_askl = accuracy_score(predictions, y)
print('Accuracy',performance_askl)

['DO' 'AT' 'DZ' ... 'CW' 'DF' 'AH']
0        DO
1        AT
2        DZ
3        EF
4        DW
         ..
49995    CK
49996    EH
49997    CW
49998    BW
49999    AH
Name: class, Length: 50000, dtype: category
Categories (100, object): ['AB', 'AC', 'AD', 'AE', ..., 'EH', 'EI', 'EJ', 'EK']
Accuracy 0.7315
[(0.300000, SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'gaussian_nb', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'select_rates_classification', 'data_preprocessor:feature_type:categorical_transformer:categorical_encoding:__choice__': 'encoding', 'data_preprocessor:feature_type:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'most_frequent', 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'normalize', 'feature_preprocessor:select_rates_classification:alpha': 0.1245

In [38]:
# estimator statistics
print(estimator_askl.sprint_statistics())

auto-sklearn results:
  Dataset name: unknown
  Metric: accuracy
  Best validation score: 0.680687
  Number of target algorithm runs: 65
  Number of successful target algorithm runs: 22
  Number of crashed target algorithm runs: 34
  Number of target algorithms that exceeded the time limit: 7
  Number of target algorithms that exceeded the memory limit: 2



In [39]:
import pandas as pd 

# read test set
data_test = pd.read_csv('data_test.csv')

# delete last column, last column contains id's which are not needed
X_test = data_test.iloc[:,:-1]

# convert object column types to category
cat_columns = X_test.select_dtypes(['object']).columns
for col in cat_columns:
    X_test[col] = X_test[col].astype('category')

# predict on the test set
predictions = estimator_askl.predict(X_test)

                 V1           V2           V3  ...         V98  V99  V100
0      10275.506530  -780.853355   161.272602  ...  558.913914    E     D
1        122.679569   450.357149    10.080290  ...  384.783132    D     D
2       4218.142745  -324.683204  -286.366977  ...  430.025897    C     D
3        -89.375203  -312.942748  -457.068004  ...  444.067717    E     D
4       6711.263646  -601.601567  -259.638255  ...  476.498303    C     D
...             ...          ...          ...  ...         ...  ...   ...
66233  -9099.126809 -1892.371657  3162.288301  ...  111.966724    F     D
66234    704.060897   -99.375285   455.823393  ...  579.461630    D     D
66235   -363.026044  -362.748056 -1225.226010  ...  582.998352    E     D
66236   1862.240866  -363.448544  -296.160499  ...  503.231474    D     D
66237  28944.148082   175.777220 -1171.402632  ...  435.696238    D     E

[66238 rows x 100 columns]


In [40]:
# create a dataframe for submission
pred_df = pd.DataFrame(predictions, columns=['Predicted'])
results = pd.concat([data_test['id'], pred_df], axis=1)
print(results)

          id Predicted
0          1        BS
1          2        AF
2          3        DK
3          4        AN
4          5        AT
...      ...       ...
66233  66234        DO
66234  66235        AN
66235  66236        BM
66236  66237        DY
66237  66238        BV

[66238 rows x 2 columns]


In [41]:
# export results to csv-file
results.to_csv('result5.csv',index=False)

In [42]:
# details of the estimator
estimator_askl.leaderboard(detailed=True)

,rank,ensemble_weight,type,cost,duration,config_id,train_loss,seed,start_time,end_time,budget,status,data_preprocessors,feature_preprocessors,balancing_strategy,config_origin
model_id,,,,,,,,,,,,,,,,
63,1,0.14,gaussian_nb,0.319313,12.821567,62,0.242816,0,1.636646e+09,1.636646e+09,0.0,StatusType.SUCCESS,[],[no_preprocessing],weighting,Local Search
48,2,0.02,gaussian_nb,0.408141,27.283537,47,0.366763,0,1.636646e+09,1.636646e+09,0.0,StatusType.SUCCESS,[],[extra_trees_preproc_for_classification],weighting,Random Search
46,3,0.04,gaussian_nb,0.520308,17.206369,45,0.448251,0,1.636646e+09,1.636646e+09,0.0,StatusType.SUCCESS,[],[select_rates_classification],none,Random Search
36,4,0.12,gaussian_nb,0.633921,7.969635,35,0.608911,0,1.636646e+09,1.636646e+09,0.0,StatusType.SUCCESS,[],[extra_trees_preproc_for_classification],weighting,Random Search (sorted)
64,5,0.12,gaussian_nb,0.638987,52.135037,63,0.522121,0,1.636646e+09,1.636646e+09,0.0,StatusType.SUCCESS,[],[fast_ica],none,Local Search
32,6,0.26,gaussian_nb,0.883401,8.382097,31,0.872667,0,1.636646e+09,1.636646e+09,0.0,StatusType.SUCCESS,[],[no_preprocessing],weighting,Random Search (sorted)
42,7,0.30,gaussian_nb,0.943645,45.561078,41,0.944531,0,1.636646e+09,1.636646e+09,0.0,StatusType.SUCCESS,[],[select_rates_classification],weighting,Random Search (sorted)


In [43]:
# print estimator accuracy
print(estimator_askl.score(X,y))

0.7315


In [44]:
# code to export trained model
import pickle

filename = 'model5.sav'
pickle.dump(estimator_askl, open(filename, 'wb'))

In [45]:
from sklearn.metrics import accuracy_score

# code to import trained model
loaded_model = pickle.load(open(filename, 'rb'))
predictions = loaded_model.predict(X)
performance_askl = accuracy_score(predictions, y)

# print statistics of the loaded model
print(loaded_model)
print(loaded_model.leaderboard())
print(performance_askl)

AutoSklearnClassifier(include={'classifier': ['random_forest', 'adaboost',
                                              'gaussian_nb', 'sgd',
                                              'gradient_boosting']},
                      memory_limit=4096, metric=accuracy, n_jobs=-1,
                      per_run_time_limit=180,
                      resampling_strategy='holdout-iterative-fit',
                      time_left_for_this_task=900)
          rank  ensemble_weight         type      cost   duration
model_id                                                         
63           1             0.14  gaussian_nb  0.319313  12.821567
48           2             0.02  gaussian_nb  0.408141  27.283537
46           3             0.04  gaussian_nb  0.520308  17.206369
36           4             0.12  gaussian_nb  0.633921   7.969635
64           5             0.12  gaussian_nb  0.638987  52.135037
32           6             0.26  gaussian_nb  0.883401   8.382097
42           7             